In [ ]:
!pip install datashader
!pip install holoviews hvplot colorcet
!pip install geoviews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 18.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib as mpl
import holoviews as hv
import geoviews as gv
import geoviews.tile_sources as gts
from bokeh.plotting import show, output_notebook
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def plot_map(df: pd.DataFrame, parameter_name: str, colormap: mpl.colors.LinearSegmentedColormap,
             title: str, point_size: int = 8, width: int = 900, height: int = 600, alpha: float = 1,
             bgcolor: str = 'white'):

    gdf = gv.Points(df, ['lon', 'lat'], [parameter_name]) # obiekt zawierający punkty
    tiles = gts.OSM # wybór mapy tła, w tym wypadku OpenStreetMap

    map_with_points = tiles * gdf.opts(
        title=title,
        color=parameter_name,
        cmap=colormap,
        size=point_size,
        width=width,
        height=height,
        colorbar=True,
        toolbar='above',
        tools=['hover', 'wheel_zoom', 'reset'],
        alpha=alpha, # przezroczystość
        bgcolor=bgcolor
    )
    map_with_points.opts(bgcolor=bgcolor)
    return hv.render(map_with_points)

# Aktualny zbiór anotowany

In [ ]:
NASA_an = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_an.csv', sep=';')

In [ ]:
NASA_an

,lon,lat,pustynia,step
0,-98.4375,49.0625,0,0
1,-84.4375,51.9375,0,0
2,-100.5625,47.0625,0,0
3,-104.6875,48.8125,0,0
4,-95.1875,36.5625,0,0
...,...,...,...,...
1190,-108.0625,42.8125,0,0
1191,-103.1875,51.5625,0,0
1192,-88.8125,51.5625,0,0
1193,-120.5625,51.3125,0,0


### Liczba terenów pustynnych, stepowych i innych oraz jaki procent całości stanowią

In [ ]:
pustynia = NASA_an['pustynia'].sum()
pustynia_procent = pustynia / len(NASA_an)

In [ ]:
print(f"Etykietę 'pustynia' otrzymało {pustynia} lokalizacji, co stanowi {pustynia_procent*100:.2f}% wszystkich zaanotowanych obszarów.")

Etykietę 'pustynia' otrzymało 159 lokalizacji, co stanowi 13.31% wszystkich zaanotowanych obszarów.


In [ ]:
step = NASA_an['step'].sum()
step_procent = step / len(NASA_an)

In [ ]:
print(f"Etykietę 'step' otrzymało {step} lokalizacji, co stanowi {step_procent*100:.2f}% wszystkich zaanotowanych obszarów.")

Etykietę 'step' otrzymało 177 lokalizacji, co stanowi 14.81% wszystkich zaanotowanych obszarów.


In [ ]:
inne = len(NASA_an) - pustynia - step
inne_procent = inne / len(NASA_an)

In [ ]:
print(f"Za żaden z rozważanych terenów zostało uznane {inne} lokalizacji, co stanowi {inne_procent*100:.2f}% wszystkich zaanotowanych obszarów.")

Za żaden z rozważanych terenów zostało uznane 859 lokalizacji, co stanowi 71.88% wszystkich zaanotowanych obszarów.


### Wizualizacja zaanotowanych punktów

In [ ]:
NASA_an_points = NASA_an.copy()
NASA_an_points['klasyfikacja'] = 'niepustynia'
NASA_an_points.loc[NASA_an_points['pustynia'] == 1, 'klasyfikacja'] = 'pustynia'
NASA_an_points.loc[NASA_an_points['step'] == 1, 'klasyfikacja'] = 'step'
output_notebook()
show(plot_map(df = NASA_an_points, parameter_name = 'klasyfikacja',
              colormap = dict(zip(['pustynia', 'niepustynia', 'step'], ['yellow', 'green', 'orange'])),
              title = "Zaanotowane punkty", point_size = 6, alpha = 0.7))

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
